In [1]:
from sklearn.datasets import fetch_20newsgroups
from collections import defaultdict
from nltk.stem import WordNetLemmatizer
from nltk.corpus import names
from nltk.stem import PorterStemmer
import numpy as np
import nltk
import pandas as pd
import os
nltk.download('names')
from sklearn.feature_extraction.text import TfidfVectorizer
ps = PorterStemmer()
all_names = names.words()
WNL = WordNetLemmatizer()

path_train= "/Users/antonis/Downloads/20news-bydate/20news-bydate-train"



def gather(path):
    df = pd.DataFrame()
    for file in os.listdir(path):
        tag = file
        for doc in os.listdir(path+'/'+file):
            docpath = path+'/'+file+'/'+doc
            f = open(docpath, "r",encoding='cp1252')
            content = f.read()
            temp = pd.DataFrame(
                {
                    'content':content,
                    'tag':tag
                },index=[0]
            )
            df = pd.concat([df, temp])



    df.content =df.content.replace(to_replace='From:(.*\n)', value='', regex=True) ##remove from to email
    df.content =df.content.replace(to_replace='lines:(.*\n)', value='', regex=True)
    df.content =df.content.replace(to_replace='Subject:(.*\n)', value='', regex=True)
    df.content =df.content.replace(to_replace='[!"#$%&\'()*+,/:;<=>?@[\\]^_`{|}~]', value=' ', regex=True) #remove punctuation except
    df.content =df.content.replace(to_replace='-', value=' ', regex=True)
    df.content =df.content.replace(to_replace='\s+', value=' ', regex=True)    #remove new line
    df.content =df.content.replace(to_replace='  ', value='', regex=True)                #remove double white space
    df.content =df.content.apply(lambda x:x.strip())  # Ltrim and Rtrim of whitespace

    df['content']=[entry.lower() for entry in df['content']]
    return df

df_news_train = gather(path_train)



def clean(data):
    cleaned = defaultdict(list)
    count = 0
    for group in data:
        for words in group.split():
            if words.isalpha() and words not in all_names:
                words = ps.stem(words)
                cleaned[count].append(WNL.lemmatize(words.lower()))
        cleaned[count] = ' '.join(cleaned[count])
        count +=1
    return(list(cleaned.values()))

x_train = clean(df_news_train['content'])


tf = TfidfVectorizer(stop_words='english', max_features=8000,use_idf=True)
tfidf = tf.fit_transform(x_train)




[nltk_data] Downloading package names to /Users/antonis/nltk_data...
[nltk_data]   Package names is already up-to-date!


In [7]:
from sklearn.feature_extraction.text import CountVectorizer

vocab = tf.vocabulary_

countvectorizer = CountVectorizer(analyzer= 'word', stop_words='english',vocabulary=vocab)

count_wm = countvectorizer.fit_transform(x_train)


In [8]:
from sklearn.feature_extraction.text import TfidfTransformer
from numpy import asarray
from numpy import savetxt


#using Tfidftransformer and count vectorizer

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True) 
tfidf_transformer.fit(count_wm)


# print idf values 
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=countvectorizer.get_feature_names(),columns=["idf_weights"]) 
 
# sort ascending 
df_idf.sort_values(by=['idf_weights'])

# count matrix 
count_vector=countvectorizer.transform(x_train) 
 
# tf-idf scores 
tf_idf_vector=tfidf_transformer.transform(count_vector)


feature_names = countvectorizer.get_feature_names() 
 
#get tfidf vector for first document 
first_document_vector=tf_idf_vector[10342] 
 
#print the scores 
df = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["tfidf"]) 
df.sort_values(by=['tfidf'],ascending=False)

#using Tfidfvectorizer

# get the first vector out (for the first document) 
first_vector_tfidfvectorizer=tfidf[0] 

# place tf-idf values in a pandas data frame
df_tfidf = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), index=tf.get_feature_names(), columns=["tfidf"]) 
df.sort_values(by=["tfidf"],ascending=False)


print(df_tfidf)

def get_key(val):
    for key, value in tf.vocabulary_.items():
         if val == value:
             return key
 
    return "key doesn't exist"




         tfidf
aa         0.0
aaa        0.0
aaron      0.0
ab         0.0
abandon    0.0
...        ...
zubov      0.0
zurich     0.0
zv         0.0
zx         0.0
zz         0.0

[8000 rows x 1 columns]


In [27]:
import json

f = open('/Users/antonis/PycharmProjects/soup/news/Huffpost.json')

data = json.load(f)

documents = []

for i in data:
    documents.append(data[i])



In [71]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import metrics
    
input=[documents[50]]

# this steps generates word counts for the words in your docs 
# word_count_vector=countvectorizer.fit_transform(input)

testvector = tf.transform(input)

distances = cosine_similarity(testvector, tfidf)
prediction = np.argmax(distances, 1)

print(df_news_train['tag'].iloc[prediction])
print(input)


0    soc.religion.christian
Name: tag, dtype: object
['Yes, government officials can’t quite agree on whether we should go or not – and advising us to avoid kissing strangers under the mistletoe if we do. But what is a festive party without any snogging? (And let’s not forget that some government officials are partial to a Chrimbo party themselves. Not to mention an illicit snog). Whether it’s colleagues who’ve been making eyes at each other in the office for weeks leading up to the big event, or just those who happened to be, well, standing next to each other, a cheeky pash between co-workers is all part of the fun. But work and pensions secretary, Thérèse Coffey, has said no more of that please. “I don’t think there should be much snogging under the mistletoe,” she told ITV’s Peston programme.“You don’t need to do things like that. But I think we should all be trying to enjoy the Christmas ahead of us.” Which has got us thinking, what else do we love (or hate) or love to hate about t

In [11]:
print(df_idf)

         idf_weights
aa          6.750366
aaa         6.837377
aaron       6.239540
ab          5.812096
abandon     6.483737
...              ...
zubov       7.848978
zurich      7.625834
zv          8.031299
zx          6.402059
zz          7.625834

[8000 rows x 1 columns]


In [73]:
path_test= "/Users/antonis/Downloads/20news-bydate/20news-bydate-test"
df_news_test = gather(path_test)


In [92]:
print(df_news_test.iloc[5400])


content    lines 29 nntp posting host acad3.alaska.edu or...
tag                                         rec.sport.hockey
Name: 0, dtype: object


In [94]:
thetest = df_news_test['content'].iloc[5400]
thetest = [thetest]
testvector = tf.transform(thetest)
distances = cosine_similarity(testvector, tfidf)
prediction = np.argmax(distances, 1)

print(prediction)
print(df_news_train['tag'].iloc[prediction])


[6698]
0    comp.windows.x
Name: tag, dtype: object
